In [1]:
import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import cv2
import keras
import numpy as np
import matplotlib.pyplot as plt
import segmentation_models as sm
sm.set_framework('keras')
import ad_seg_utils as seg_utils
import bb_eval_utils as eval_utils
import csv
import math
import sys

Using TensorFlow backend.


Segmentation Models: using `keras` framework.


In [2]:
# This function is to output the correct perturbation name in the files as the namings are kindof chaos...
def Perturb_file_name(perturbation):
    name = ""
    if perturbation == "ElasticTransform":
        name = "elastic"
    elif perturbation == "GaussianNoise":
        name = "noise"
    else:
        name = perturbation.lower()
    
    return name
        
    

In [3]:
def main():
    ################################ Model loading (this part will be replaced with new data type soon) ################################
    SEGMENTATION = "skin"
    PERTURBATION = "Fog"
    Perturbation_file_name = Perturb_file_name(PERTURBATION)
    print(Perturbation_file_name)
    MODEL_PREFIX = "severity_3"
    Severity = MODEL_PREFIX[-1]
    print(Severity)
    MODEL_NAME = "/skin_base/base_skin_LR1.h5"

    print("Program initiating... type of segmentation: " + SEGMENTATION + ", type of perturbation: " + PERTURBATION)

    BIN_SEG = True
    CLASSES = ['background', 'skin']
    WEIGHTS = np.array([1, 1])
    target_idx = 1
    
    BACKBONE = 'efficientnetb3'
    preprocess_input = sm.get_preprocessing(BACKBONE)

    # set parameters based on the type of segmentation
    if SEGMENTATION == 'SKIN' or SEGMENTATION == 'skin':
        pass
    elif SEGMENTATION == 'AD' or SEGMENTATION == 'ad':
        BIN_SEG = False
        target_idx = 2
        CLASSES = ['background', 'skin', 'eczema']
        WEIGHTS = np.array([1, 1, 1])
    else:
        print('Unexpected type of segmentation, should be either skin or ad\n program terminated')
        return -1
    
    """# Model Evaluation"""
    PROJ_DIR = "/rds/general/user/ym521/home/EczemaNet-DeepLearning-Segmentation-master"
    OUTPUT_DIR = os.path.join(PROJ_DIR, 'output_trained_on_SWET')
    PRED_DIR = os.path.join(OUTPUT_DIR, 'predictions/' + SEGMENTATION + '_' + MODEL_PREFIX + '_' + PERTURBATION + '/crops')
    BB_DIR = os.path.join(OUTPUT_DIR, 'predictions/' + SEGMENTATION + '_' + MODEL_PREFIX + '_' + PERTURBATION + '/boxes')
    EVAL_DIR = os.path.join(OUTPUT_DIR, 'evaluations')
    MODEL_DIR = os.path.join(OUTPUT_DIR, 'models')
    DATA_DIR = os.path.join(PROJ_DIR, 'data')

    # new dataset
    x_ref_dir = os.path.join(DATA_DIR, 'dataset_SWET/test_set_SWET/reals')
    y_ref_dir = os.path.join(DATA_DIR, 'dataset_SWET/test_set_SWET/labels')
    x_perturb_dir = os.path.join(DATA_DIR, 'perturbed_dataset_SWET/' + PERTURBATION + '/' + MODEL_PREFIX + '/reals')
    y_perturb_dir = os.path.join(DATA_DIR, 'perturbed_dataset_SWET/' + PERTURBATION + '/' + MODEL_PREFIX + '/labels')
    #     x_perturb_dir = '/rds/general/user/ym521/projects/eczemanet/live/RoI_datasets_RM/perturbed_test_sets/adversarial_test_set_' + PERTURBATION
    print('reading ref images from: ' + str(x_ref_dir))
    print('reading perturbed images from: ' + str(x_perturb_dir))

    reference_dataset = seg_utils.Dataset(
        x_ref_dir,
        y_ref_dir,
        classes=CLASSES,
        augmentation=None,
        preprocessing=seg_utils.get_preprocessing(preprocess_input),
        is_train=False,
        use_full_resolution=False,
        binary_seg=BIN_SEG,
    )

    perturbed_dataset = seg_utils.Dataset(
        x_perturb_dir,
        y_ref_dir,
        classes=CLASSES, 
        augmentation=None,
        preprocessing=seg_utils.get_preprocessing(preprocess_input),
        is_train=False,
        use_full_resolution=False,
        binary_seg=BIN_SEG,
        PERTURBATION=Perturbation_file_name,
        SEVERITY=Severity,
    )

    model = seg_utils.load_model(dir=MODEL_DIR + MODEL_NAME, classes=CLASSES, weights=WEIGHTS)
    print('Trained model loaded!')

    print('Creating directories and clearing previous masks...')
    os.system("mkdir -p " + PRED_DIR)
    os.system("mkdir -p " + BB_DIR)
    os.system("rm " + PRED_DIR + "/*.png")
    os.system("rm " + PRED_DIR + "/*.PNG")
    os.system("rm " + BB_DIR + "/*.png")
    os.system("rm " + BB_DIR + "/*.PNG")
    os.system("rm " + EVAL_DIR + "/robustness_evaluation_" + SEGMENTATION + "_" + MODEL_PREFIX + "_" + PERTURBATION + ".csv")
    print('Done! Now saving new prediction masks...')

    # create a list to store a series of IoU values
    iou = []

    with open(EVAL_DIR + "/robustness_evaluation_" + SEGMENTATION + "_" + MODEL_PREFIX + "_" + PERTURBATION + ".csv", 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Reference file name", "Perturbation file name", "IoU"])
        for i in range(len(reference_dataset)):
            # prediction for reference and perturbed images
            ref_image, gt_mask = reference_dataset[i]
            ref_image = np.expand_dims(ref_image, axis=0)
            ref_pred = model.predict(ref_image)

            perturbed_image, _ = perturbed_dataset[i]
            perturbed_image = np.expand_dims(perturbed_image, axis=0)
            perturbed_pred = model.predict(perturbed_image)

            # extract the last index to decide which mask to output. [0: background; 1: skin; 2: eczema]
            ref_pred_img = ref_pred[0, :, :, target_idx]
            ref_pred_img = (ref_pred_img * 255).astype(np.uint8)

            perturbed_pred_img = perturbed_pred[0,:,:,target_idx]
            perturbed_pred_img = (perturbed_pred_img * 255).astype(np.uint8)

            # save the images
            cv2.imwrite(os.path.join(PRED_DIR, "pred_" + reference_dataset.images_ids[i]), ref_pred_img)
            cv2.imwrite(os.path.join(PRED_DIR, "pred_" + perturbed_dataset.images_ids[i]), perturbed_pred_img)
            print("saving", reference_dataset.images_ids[i])
            print("saving", perturbed_dataset.images_ids[i])

            # generate bounding boxes for each predicted mask
            perturbed_boxes = []
            perturbed_pred_img = cv2.imread(os.path.join(PRED_DIR, "pred_" + perturbed_dataset.images_ids[i]))
            gray = cv2.cvtColor(perturbed_pred_img,cv2.COLOR_BGR2GRAY)
            thresh = cv2.threshold(gray,127,255,cv2.THRESH_BINARY)[1]
            perturbed_contours = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            perturbed_contours = perturbed_contours[0] if len(perturbed_contours) == 2 else perturbed_contours[1]

            ref_boxes = []
            ref_pred_img = cv2.imread(os.path.join(PRED_DIR, "pred_" + reference_dataset.images_ids[i]))
            gray = cv2.cvtColor(ref_pred_img, cv2.COLOR_BGR2GRAY)
            thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)[1]
            ref_contours = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            ref_contours = ref_contours[0] if len(ref_contours) == 2 else ref_contours[1]

            # extract contours from perturbed predictions
            for cntr in perturbed_contours:
                rect = cv2.minAreaRect(cntr)
                box = cv2.boxPoints(rect)
                box = np.int0(box)
                area = cv2.contourArea(cntr)
                # Abandon boxes with too small area
                if seg_utils.crop_filter(area):
                    perturbed_boxes.append(box)

            # extract contours from reference predictions
            for cntr in ref_contours:
                rect = cv2.minAreaRect(cntr)
                box = cv2.boxPoints(rect)
                box = np.int0(box)
                area = cv2.contourArea(cntr)
                # Abandon boxes with too small area
                if seg_utils.crop_filter(area):
                    ref_boxes.append(box)

            # append IoU to the list
            iou_per_image = eval_utils.compute_robustness_iou(gt_mask, ref_boxes, perturbed_boxes)
            iou.append(iou_per_image)
            writer.writerow([reference_dataset.images_ids[i], perturbed_dataset.images_ids[i], iou_per_image])

        # Append the mean performance to the end of csv
        writer.writerow(['mean', '', np.mean(iou)])
        writer.writerow(['se', '', np.std(iou) / np.sqrt(len(iou))])
        writer.writerow(['degradation', '', 1 - np.mean(iou)])

        print('Done!')

if __name__ == "__main__":
    main()


fog
3
Program initiating... type of segmentation: skin, type of perturbation: Fog
reading ref images from: /rds/general/user/ym521/home/EczemaNet-DeepLearning-Segmentation-master/data/dataset_SWET/test_set_SWET/reals
reading perturbed images from: /rds/general/user/ym521/home/EczemaNet-DeepLearning-Segmentation-master/data/perturbed_dataset_SWET/Fog/severity_3/reals
['01001_wk00_1.png', '01001_wk00_2.png', '01001_wk04_2.png', '01001_wk12_1.png', '01001_wk12_2.png', '01001_wk16.png', '01003_wk12.png', '01003_wk16.png', '01003_wk16_2.png', '01005_wk00.png', '01005_wk00_2.png', '01005_wk12.png', '01005_wk12_2.png', '01005_wk16.png', '01005_wk16_1.png', '01011_wk04.png', '01011_wk04_2.png', '01011_wk12.png', '01011_wk12_2.png', '01013_wk00.png', '01013_wk00_2.png', '01013_wk04.png', '01013_wk04_2.png', '01013_wk12.png', '01014_wk04.png', '01014_wk12.png', '01014_wk12_2.png', '01014_wk16.png', '01029_wk00.png', '01029_wk04.png', '01029_wk12.png', '01029_wk12_1.png', '01029_wk16.png', '01030

2022-09-06 20:49:43.718500: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-09-06 20:49:44.297462: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-09-06 20:49:44.297539: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (cx3-15-29.cx3.hpc.ic.ac.uk): /proc/driver/nvidia/version does not exist
2022-09-06 20:49:44.299193: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2022-09-06 20:49:44.343078: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2250080000 Hz
2022-09-06 20:49:44.366179: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5639758e4590 initialized for platform Host (this does not guarantee that XLA will

Trained model loaded!
Creating directories and clearing previous masks...


sh: which: line 1: syntax error: unexpected end of file
sh: error importing function definition for `which'
sh: module: line 1: syntax error: unexpected end of file
sh: error importing function definition for `module'
sh: scl: line 1: syntax error: unexpected end of file
sh: error importing function definition for `scl'
sh: ml: line 1: syntax error: unexpected end of file
sh: error importing function definition for `ml'
sh: which: line 1: syntax error: unexpected end of file
sh: error importing function definition for `which'
sh: module: line 1: syntax error: unexpected end of file
sh: error importing function definition for `module'
sh: scl: line 1: syntax error: unexpected end of file
sh: error importing function definition for `scl'
sh: ml: line 1: syntax error: unexpected end of file
sh: error importing function definition for `ml'
sh: which: line 1: syntax error: unexpected end of file
sh: error importing function definition for `which'
sh: module: line 1: syntax error: unexpected

Done! Now saving new prediction masks...

saving 01001_wk00_1.jpg
saving 01001_wk00_1_fog_i3_aug.png
IoU:  0.9094581690288442
saving 01001_wk00_2.jpg
saving 01001_wk00_2_fog_i3_aug.png
IoU:  0.8317529416835481
saving 01001_wk04_2.jpg
saving 01001_wk04_2_fog_i3_aug.png
IoU:  1.0
saving 01001_wk12_1.jpg
saving 01001_wk12_1_fog_i3_aug.png
IoU:  0.838788409645095
saving 01001_wk12_2.jpg
saving 01001_wk12_2_fog_i3_aug.png
IoU:  0.7949944572204586
saving 01001_wk16.jpg
saving 01001_wk16_fog_i3_aug.png
IoU:  0.7476501645476715
saving 01003_wk12.jpg
saving 01003_wk12_fog_i3_aug.png
IoU:  0.9962489780869789
saving 01003_wk16.jpg
saving 01003_wk16_fog_i3_aug.png
IoU:  0.9877190456428021
saving 01003_wk16_2.jpg
saving 01003_wk16_2_fog_i3_aug.png
IoU:  0.9953111888454765
saving 01005_wk00.jpg
saving 01005_wk00_fog_i3_aug.png
IoU:  0.7579766836356054
saving 01005_wk00_2.jpg
saving 01005_wk00_2_fog_i3_aug.png
IoU:  0.8686207722192615
saving 01005_wk12.jpg
saving 01005_wk12_fog_i3_aug.png
IoU:  0.926

KeyboardInterrupt: 